In [1]:
import pandas as pd
import pickle

In [2]:
# Imports
import numpy as np
import matplotlib
from matplotlib import pyplot as plt
%matplotlib inline     
## use `%matplotlib notebook` for interactive figures
# plt.style.use('ggplot')
import sklearn

import tigramite
from tigramite import data_processing as pp
from tigramite.toymodels import structural_causal_processes as toys

from tigramite import plotting as tp
from tigramite.pcmci import PCMCI

from tigramite.independence_tests.parcorr import ParCorr
from tigramite.independence_tests.robust_parcorr import RobustParCorr
from tigramite.independence_tests.parcorr_wls import ParCorrWLS
from tigramite.independence_tests.gpdc import GPDC
from tigramite.independence_tests.cmiknn import CMIknn
from tigramite.independence_tests.cmisymb import CMIsymb
from tigramite.independence_tests.gsquared import Gsquared
from tigramite.independence_tests.regressionCI import RegressionCI
from pandas import read_pickle
import plotly.express as px

In [3]:
close_prices = read_pickle('/Users/georgii_nigm/Applications/VS Code projects/academic_causal_portfolio/cleaned/close_prices_fed.pkl')

In [4]:
close_prices_crop = close_prices.iloc[close_prices_crop.index >'2000-01-01',10:]

NameError: name 'close_prices_crop' is not defined

In [ ]:
close_prices_crop

In [ ]:
var_names=close_prices_crop.columns
dataframe = pp.DataFrame(close_prices_crop.values, var_names=var_names)
tp.plot_timeseries(dataframe)

# Stationarizing

In [ ]:
df_train = close_prices_crop

In [ ]:
from statsmodels.tsa.stattools import adfuller
from statsmodels.tsa.stattools import kpss

def adf_test(df):
    print('if p>0,5 - than ts is non-stationary\n')
    
    for stock in df.columns:
        result = adfuller(df[stock].values)
        if result[1] < 0.05:
            print(f'{stock}: stationary. p-value: %f' % result[1])
        if result[1] >= 0.05:
            print(f'{stock}: Non-Stationary. p-value: %f' % result[1])


    # print('ADF Statistics: %f' % result[0])
    # print('p-value: %f' % result[1])
    # print('Critical values:')
    # for key, value in result[4].items():
    #     print('\t%s: %.3f' % (key, value))


def kpss_test(df):    
    statistic, p_value, n_lags, critical_values = kpss(df.values)
    
    print(f'KPSS Statistic: {statistic}')
    print(f'p-value: {p_value}')
    print(f'num lags: {n_lags}')
    print('Critial Values:')
    for key, value in critical_values.items():
        print(f'   {key} : {value}')

In [ ]:
adf_test(close_prices_crop)

In [ ]:
df_train_transformed = df_train#.diff().dropna()

In [ ]:
adf_test(df_train_transformed)

# VAR

In [ ]:
from statsmodels.tsa.api import VAR

In [ ]:
# model = VAR(df_train_transformed)
# for i in [2*x for x in [1,2,3,4,5,6,7,8,9,10,11,12,13,14,15]]:
#     result = model.fit(i)
#     print('Lag Order =', i)
#     print('AIC : ', result.aic)
#     print('BIC : ', result.bic)
#     print('FPE : ', result.fpe)
#     print('HQIC: ', result.hqic, '\n')

In [ ]:
results = model.fit(maxlags=1, ic='aic')
results.summary()

In [ ]:
results.resid_corr

In [ ]:
from statsmodels.stats.stattools import durbin_watson

out = durbin_watson(results.resid)

for col, val in zip(df_train_transformed.columns, out):
    print(col, ':', round(val, 2))

In [ ]:
from statsmodels.tsa.stattools import grangercausalitytests

maxlag=15
test = 'ssr_chi2test'

def grangers_causation_matrix(data, variables, test='ssr_chi2test', verbose=False):    
   
    df = pd.DataFrame(np.zeros((len(variables), len(variables))), columns=variables, index=variables)
    for c in df.columns:
        for r in df.index:
            test_result = grangercausalitytests(data[[r, c]], maxlag=maxlag, verbose=False)
            p_values = [round(test_result[i+1][0][test][1],4) for i in range(maxlag)]
            if verbose: print(f'Y = {r}, X = {c}, P Values = {p_values}')
            min_p_value = np.min(p_values)
            df.loc[r, c] = min_p_value
    df.columns = [var + '_x' for var in variables]
    df.index = [var + '_y' for var in variables]
    return df

grangers_causation_matrix(df_train_transformed, variables = df_train_transformed.columns)